In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('vg_sales_ratings.csv')
data.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


The column values are analysed using the pd.info() method. It gives details about each column's datatypes and also the number of non-null values.

In [3]:
print(data.shape, data.columns)

(16719, 16) Index(['Name', 'Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Critic_Score',
       'Critic_Count', 'User_Score', 'User_Count', 'Developer', 'Rating'],
      dtype='object')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


`User_Score` should be float, but is showing as object

In [5]:
data.User_Score.unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

Similarly `Rating` should be numeric to be usable for prediction

In [6]:
data.Rating.unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

### What is ESRB rating?

There are 5 ratings.

- The original ratings were RP, EC, K-A, T, M, and AO. 

- K-A was replaced by E in January 1st, 1998. 

- They added an E10+ rating in March 2nd, 2005

Mapping I have used

- EC is rated as A (5)

- E rated video games are rated A (5).

- E10+ rated video games are rated B (4).

- T rated video games are rated B15 (3).

- M rated video games are rated C (2).

- AO rated video games are rated D (1).

- RP/RP Likely M rated video games are rated (2)

In [7]:
rating_mapping = {'EC': 5, 'E': 5, 'E10+': 4, 'T': 3, 'M': 2, 'AO': 1, 'RP': 2, 'K-A': 5, 0: 0}

data['Rating'].fillna(0, inplace=True)
data['Rating'] = data['Rating'].apply(lambda x: rating_mapping[x])

In [8]:
data[data['User_Score'] == 'tbd'] = 0

Descriptive analysis of the dataset is performed using the pd.describe() method

In [9]:
data.describe()

,Year_of_Release,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Count,Rating
count,16498.000000,16719.000000,16719.000000,16719.000000,16719.000000,16719.000000,9480.000000,9480.000000,10015.000000,16719.000000
mean,1711.419263,0.245039,0.138715,0.077207,0.045031,0.506176,52.267932,21.350949,122.948078,1.620133
std,710.470595,0.814905,0.503002,0.308755,0.186586,1.551421,32.903314,20.737473,493.536305,1.936862
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1999.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,4.000000,0.000000
50%,2006.000000,0.040000,0.010000,0.000000,0.010000,0.130000,66.000000,17.000000,14.000000,0.000000
75%,2010.000000,0.210000,0.100000,0.030000,0.030000,0.440000,78.000000,33.000000,51.000000,3.000000
max,2017.000000,41.360000,28.960000,10.220000,10.570000,82.530000,98.000000,113.000000,10665.000000,5.000000


Data seems to be missing in some columns

In [10]:
data.isna().sum()

Name                  2
Platform              0
Year_of_Release     221
Genre                 2
Publisher            52
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       7239
Critic_Count       7239
User_Score         6704
User_Count         6704
Developer          6601
Rating                0
dtype: int64

In [11]:
cleaned_data = data.drop(['Critic_Count', 'User_Count', 'Developer'], axis=1)
cleaned_data.isna().sum()

Name                  2
Platform              0
Year_of_Release     221
Genre                 2
Publisher            52
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       7239
User_Score         6704
Rating                0
dtype: int64

In [12]:
platforms = cleaned_data[cleaned_data['Year_of_Release'].isna()].Platform.unique()
platforms

array(['PS2', 'Wii', '2600', 'X360', 'PC', 'PS3', 'PS', 'PSP', 'XB', 'GB',
       'GBA', 'DS', 'GC', '3DS', 'N64', 'PSV'], dtype=object)

In [13]:
values = {}

for platform in platforms:
    values[platform] = int(cleaned_data[cleaned_data['Platform'] == platform]['Year_of_Release'].median())

values

{'PS2': 2004,
 'Wii': 2009,
 '2600': 1982,
 'X360': 2010,
 'PC': 2010,
 'PS3': 2011,
 'PS': 1998,
 'PSP': 2009,
 'XB': 2004,
 'GB': 1997,
 'GBA': 2003,
 'DS': 2008,
 'GC': 2003,
 '3DS': 2013,
 'N64': 1999,
 'PSV': 2014}

In [14]:
def year_filler(row):
    if pd.isna(row.Year_of_Release):
        return values[row.Platform]
    
    return row.Year_of_Release

cleaned_data.Year_of_Release = cleaned_data.apply(year_filler, axis=1)

cleaned_data.Year_of_Release.isna().sum()

0

In [15]:
cleaned_data.Critic_Score.fillna(cleaned_data.Critic_Score.median(), inplace=True)
cleaned_data.User_Score.fillna(cleaned_data.User_Score.median(), inplace=True)

cleaned_data.isna().sum()

Name                2
Platform            0
Year_of_Release     0
Genre               2
Publisher          52
NA_Sales            0
EU_Sales            0
JP_Sales            0
Other_Sales         0
Global_Sales        0
Critic_Score        0
User_Score          0
Rating              0
dtype: int64

In [16]:
cleaned_data.dropna(subset=['Publisher'], inplace=True)
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16667 entries, 0 to 16718
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16665 non-null  object 
 1   Platform         16667 non-null  object 
 2   Year_of_Release  16667 non-null  float64
 3   Genre            16665 non-null  object 
 4   Publisher        16667 non-null  object 
 5   NA_Sales         16667 non-null  float64
 6   EU_Sales         16667 non-null  float64
 7   JP_Sales         16667 non-null  float64
 8   Other_Sales      16667 non-null  float64
 9   Global_Sales     16667 non-null  float64
 10  Critic_Score     16667 non-null  float64
 11  User_Score       16667 non-null  object 
 12  Rating           16667 non-null  int64  
dtypes: float64(7), int64(1), object(5)
memory usage: 1.8+ MB


In [17]:
cleaned_data.Year_of_Release = cleaned_data.Year_of_Release.astype('int64')